<a href="https://colab.research.google.com/github/lcipolina/escher/blob/master/ML/Colab-co-mod-gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab-co-mod-gan

Original repo: [zsyzzsoft/co-mod-gan](https://github.com/zsyzzsoft/co-mod-gan)

My fork: [styler00dollar/Colab-co-mod-gan](https://github.com/styler00dollar/Colab-co-mod-gan)




In [ ]:
!nvidia-smi

In [ ]:
# @title install

!git clone https://github.com/zsyzzsoft/co-mod-gan
#!git clone styler00dollar/Colab-co-mod-gan
# tensorflow 1.15 to avoid tensorflow.python.framework.errors_impl.NotFoundError
!pip install tensorflow==1.15
%cd /content/co-mod-gan

# Testing

Just place ``/content/input.png`` and ``/content/mask.png``.

In [ ]:
#@title download pretrain
!gdown --id 1dJa3DRWIkx6Ebr8Sc0v1FdvWf6wkd010 # co-mod-gan-places2-050000.pkl
#!gdown --id 1b3XxfAmJ9k2vd73j-3nPMr_lvNMQOFGE # co-mod-gan-ffhq-9-025000.pkl
#!gdown --id 1M2dSxlJnCFNM6LblpB2nQCnaimgwaaKu # co-mod-gan-ffhq-10-025000.pkl

In [ ]:
#@title (optional) read / write / resize file with opencv / invert mask
#@markdown You need masks where ``white = original`` and ``black = inpaint``. Both images are rgb and have the needed dimensions.
import cv2
path_image = '/content/input.png'
path_mask = '/content/mask.png'
image = cv2.imread(path_image)
image = cv2.resize(image, (512,512), cv2.INTER_NEAREST)
cv2.imwrite(path_image, image)

image = cv2.imread(path_mask)
image = 255-image
image = cv2.resize(image, (512,512), cv2.INTER_NEAREST)
cv2.imwrite(path_mask, image)

In [ ]:
!pip install --upgrade --no-cache-dir gdown
#!gdown --id 1YbSxWraGrQzD7UVOyPF95FCgRz_H03uD # ours
!gdown --id 1dJa3DRWIkx6Ebr8Sc0v1FdvWf6wkd010 #places2


In [ ]:
%cd /content/co-mod-gan
!python run_generator.py -c /content/co-mod-gan/co-mod-gan-places2-050000.pkl -i /content/co-mod-gan/imgs/example_image.jpg -m /content/co-mod-gan/imgs/example_mask.jpg -o /content/output.png

In [ ]:
'''
%cd /content/co-mod-gan
!python run_generator.py -c /content/network-snapshot-050060.pkl -i /content/co-mod-gan/imgs/example_image.jpg -m /content/co-mod-gan/imgs/example_mask.jpg -o /content/output.png
'''

# Training

Warnings: Free colab does not have enough RAM. You need Colab Pro with 25GB RAM. Needs ~13GB RAM. Also needs ~10GB VRAM.

Training can also crash, because of:
```  
File "/content/co-mod-gan/metrics/inception_discriminative_score.py", line 30, in _evaluate
    inception = misc.load_pkl('https://drive.google.com/uc?id=1MzTY44rLToO5APn8TZmfR7_ENSe5aZUn') # inception_v3_features.pkl
  File "/content/co-mod-gan/training/misc.py", line 29, in load_pkl
    with open_file_or_url(file_or_url) as file:
  File "/content/co-mod-gan/training/misc.py", line 25, in open_file_or_url
    return dnnlib.util.open_url(file_or_url, cache_dir='.stylegan2-cache')
  File "/content/co-mod-gan/dnnlib/util.py", line 383, in open_url
    raise IOError("Google Drive download quota exceeded -- please try again later")
OSError: Google Drive download quota exceeded -- please try again later
```

In [ ]:
%cd /content/co-mod-gan/dataset_tools
!python create_from_images.py --tfrecord-dir /content/tfrecord_dir --val-image-dir /content/val --train-image-dir /content/train --resolution 512 --num-channels 3 --num-processes 4

In [ ]:
%cd /content/co-mod-gan
# Example: /content/tfrecord_dir/tfrecord_dir-r09.tfrecords
!python run_training.py --data-dir=/content/ --dataset=tfrecord_dir --metrics=ids10k --mirror-augment True --num-gpus=1